In [ ]:
import os
import time
import copy

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from timm import create_model
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import os
import requests
import zipfile
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import defaultdict
import random
import medmnist

from torch.utils.data import Subset

import torch
import torch.nn as nn
import torch.optim as optim
from torch.linalg import inv
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import TensorDataset
from torch.cuda.amp import autocast

from torchvision import transforms
from torchvision.transforms import AutoAugment, AutoAugmentPolicy
from PIL import Image

from transformers import ViTModel, ViTImageProcessor

import timm
from timm import create_model

from torch_geometric.nn import LabelPropagation
from torch_geometric.utils import dense_to_sparse

import numpy as np
import pandas as pd

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from torch_geometric.utils import to_networkx
from torch_geometric.data import Data

from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from dppy.finite_dpps import FiniteDPP
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE

import os
import requests
import zipfile
from tqdm import tqdm
from collections import defaultdict, Counter
import random
import matplotlib.pyplot as plt
from torch.linalg import inv
import seaborn as sns
import medmnist

from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import LabelEncoder

import community
import community.community_louvain as community
import networkx as nx
from sklearn.neighbors import kneighbors_graph

In [ ]:
path = '/home/dime/Desktop/Thesis/dermamnist_224'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

num_classes = 7
num_epochs = 100
img_size=224
initial_learning_rate = 0.001
learning_rate = 0.001

In [ ]:
def plot_metrics(metrics_log):
    epochs = range(1, len(metrics_log["train_loss"]) + 1)

    plt.figure()
    plt.plot(epochs, metrics_log["train_loss"], label="Train Loss")
    plt.plot(epochs, metrics_log["val_loss"], label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Loss vs Epochs")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(epochs, metrics_log["train_accuracy"], label="Train Accuracy")
    plt.plot(epochs, metrics_log["val_accuracy"], label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs Epochs")
    plt.legend()
    plt.show()

    plt.figure()
    plt.plot(epochs, metrics_log["train_f1"], label="Train F1 Score")
    plt.plot(epochs, metrics_log["val_f1"], label="Validation F1 Score")
    plt.xlabel("Epochs")
    plt.ylabel("F1 Score")
    plt.title("F1 Score vs Epochs")
    plt.legend()
    plt.show()

def plot_class_distribution(class_counts, title):
    classes = list(class_counts.keys())
    counts = list(class_counts.values())
    
    plt.figure(figsize=(6, 3))
    plt.bar(classes, counts, color='skyblue')
    plt.xlabel('Classes')
    plt.ylabel('Counts')
    plt.title(title)
    plt.show()

In [ ]:
class NumpyDataset(Dataset):
    def __init__(self, images_path: str, labels_path: str, transform=None):
        self.images = np.load(images_path)
        self.labels = np.load(labels_path).flatten()
        self.transform = transform

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, idx: int):
        img = self.images[idx]
        img = (img * 255).astype(np.uint8)
        if img.ndim == 2:
            img = np.stack([img] * 3, axis=0)
        else:
            img = img.transpose(2, 0, 1)
        img = torch.tensor(img, dtype=torch.float32)
        if self.transform:
            img = self.transform(img)
        lbl = int(self.labels[idx])
        return img, lbl
    
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomResizedCrop(img_size, scale=(0.8,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225]),
])

def load_datasets(img_size=img_size, batch_size=batch_size, shuffle_train=True):
    train_dataset = NumpyDataset(
        os.path.join(path, 'train_images.npy'),
        os.path.join(path, 'train_labels.npy'),
        train_transform
    )
    val_dataset = NumpyDataset(
        os.path.join(path, 'val_images.npy'),
        os.path.join(path, 'val_labels.npy'),
        val_test_transform
    )
    test_dataset = NumpyDataset(
        os.path.join(path, 'test_images.npy'),
        os.path.join(path, 'test_labels.npy'),
        val_test_transform
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle_train)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

train_class_counts = compute_class_counts(os.path.join(path, 'train_labels.npy'))
val_class_counts = compute_class_counts(os.path.join(path, 'val_labels.npy'))
test_class_counts = compute_class_counts(os.path.join(path, 'test_labels.npy'))

print('Train class counts:', train_class_counts)
print('Val class counts:', val_class_counts)
print('Test class counts:', test_class_counts)

train_class_counts = compute_class_counts(os.path.join(path, 'train_labels.npy'))
total_train = sum(train_class_counts.values())
#  higher weight for less frequent classes
class_weights = [total_train / train_class_counts[i] for i in range(num_classes)]
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
print("Class weights:", class_weights)

plot_class_distribution(train_class_counts, 'Train Class Distribution')
plot_class_distribution(val_class_counts, 'Validation Class Distribution')
plot_class_distribution(test_class_counts, 'Test Class Distribution')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
model = model.to(device)

for param in model.parameters():
    param.requires_grad = True

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=initial_learning_rate, weight_decay=1e-4)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
scaler = torch.cuda.amp.GradScaler()

def train_model(model, train_loader, val_loader, device, optimizer, scheduler, scaler, 
                num_epochs=num_epochs, accumulation_steps=4, patience=5):
    metrics_log = {
        "train_loss": [],
        "train_accuracy": [],
        "train_f1": [],
        "val_loss": [],
        "val_accuracy": [],
        "val_f1": []
    }
    
    model.to(device)
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_labels, all_preds = [], []
        
        optimizer.zero_grad(set_to_none=True)
        
        for step, (images, labels) in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}")):
            images, labels = images.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels) / accumulation_steps
            
            scaler.scale(loss).backward()
            
            if (step + 1) % accumulation_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
            
            running_loss += loss.item() * accumulation_steps
            _, predicted = outputs.max(1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
        
        if (step + 1) % accumulation_steps != 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        torch.cuda.empty_cache()
        train_accuracy = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds, average='weighted')
        train_loss = running_loss / len(train_loader)
        
        model.eval()
        val_loss = 0.0
        val_labels, val_preds = [], []
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validating"):
                images, labels = images.to(device), labels.to(device)
                with torch.cuda.amp.autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = outputs.max(1)
                val_labels.extend(labels.cpu().numpy())
                val_preds.extend(predicted.cpu().numpy())
        
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average='weighted')
        val_loss /= len(val_loader)
        
        metrics_log["train_loss"].append(train_loss)
        metrics_log["train_accuracy"].append(train_accuracy)
        metrics_log["train_f1"].append(train_f1)
        metrics_log["val_loss"].append(val_loss)
        metrics_log["val_accuracy"].append(val_accuracy)
        metrics_log["val_f1"].append(val_f1)
        
        scheduler.step(val_loss)
        
        print(f"Epoch [{epoch+1}/{num_epochs}]:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}")
        print(f"  Val Loss:   {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), "vgg_derma_best.pth")
            print("Validation loss improved. Model saved.")
        else:
            patience_counter += 1
            print(f"No improvement in validation loss for {patience_counter} epoch(s).")
        
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break
    
    print(f"Final Train Loss:     {metrics_log['train_loss'][-1]:.4f}")
    print(f"Final Train Accuracy: {metrics_log['train_accuracy'][-1]:.4f}")
    print(f"Final Train F1:       {metrics_log['train_f1'][-1]:.4f}")
    print(f"Final Val Loss:       {metrics_log['val_loss'][-1]:.4f}")
    print(f"Final Val Accuracy:   {metrics_log['val_accuracy'][-1]:.4f}")
    print(f"Final Val F1:         {metrics_log['val_f1'][-1]:.4f}")
    
    plot_metrics(metrics_log)
    return metrics_log

In [ ]:
def run_experiments(seeds=[42, 43, 45], num_epochs=num_epochs, learning_rate=learning_rate, batch_size=batch_size, img_size=img_size):
    results = []
    
    for seed in seeds:
        print(f"\n\n======================= SEED {seed} =======================")
        set_seed(seed)
        start_time = time.time()
        train_loader, val_loader, test_loader = load_datasets(img_size, batch_size)
        
        model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
        model = model.to(device)
        for param in model.parameters():
            param.requires_grad = True
        
        metrics_log = train_model(model, train_loader, val_loader, device,
                                  optimizer, scheduler, scaler,
                                  num_epochs=num_epochs)
        
        final_train_acc = metrics_log["train_accuracy"][-1]
        final_train_f1  = metrics_log["train_f1"][-1]
        final_val_acc   = metrics_log["val_accuracy"][-1]
        final_val_f1    = metrics_log["val_f1"][-1]
        
        results.append((final_train_acc, final_train_f1, final_val_acc, final_val_f1))
    
    results = np.array(results)
    means = results.mean(axis=0)
    stds  = results.std(axis=0)
    end_time = time.time()
    print("\n\n======================= SUMMARY =======================")
    print(f"Seeds used: {seeds}")
    print(f"Train Accuracy: mean={means[0]:.4f}, std={stds[0]:.4f}")
    print(f"Train F1:       mean={means[1]:.4f}, std={stds[1]:.4f}")
    print(f"Val Accuracy:   mean={means[2]:.4f}, std={stds[2]:.4f}")
    print(f"Val F1:         mean={means[3]:.4f}, std={stds[3]:.4f}")
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    return (means, stds)